In [1]:
import numpy as np
from glob import glob
import cv2, os, random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [3]:
path='dogs-vs-cats/train'

In [4]:
## used for resize and in our model
ROW, COL = 96, 96

dogs, cats = [], []
y_dogs, y_cats = [], []

In [5]:
## Definition to load all our dog images
def load_dogs():
    print('Loading all dog images\n')
    dog_path = os.path.join(path, 'dog*')
    for dog_img in glob(dog_path):
        dog = cv2.imread(dog_img)
        dog = cv2.cvtColor(dog, cv2.COLOR_BGR2GRAY)
        dog = cv2.resize(dog, (ROW, COL))
        dog = image.img_to_array(dog)
        dogs.append(dog)
    print('All dog images loaded')

In [6]:
## Definition to load all our cat images
def load_cats():
    print('Loading all cat images\n')
    cat_path = os.path.join(path, 'cat*')
    for cat_img in glob(cat_path):
        cat = cv2.imread(cat_img)
        cat = cv2.cvtColor(cat, cv2.COLOR_BGR2GRAY)
        cat = cv2.resize(cat, (ROW, COL))
        cat = image.img_to_array(cat)
        cats.append(cat)
    print('All cat images loaded')

In [7]:
load_dogs()

Loading all dog images

All dog images loaded


In [8]:
load_cats()

Loading all cat images

All cat images loaded


In [9]:
classes = ['dog', 'cat']

In [10]:
## just change the labels for 0 and  1
y_dogs = [1 for item in enumerate(dogs)]
y_cats = [0 for item in enumerate(cats)]

In [11]:
## converting everything to Numpy array to fit in our model
## them creating a X and target file like we used to see
## in Machine and Deep Learning models
dogs = np.asarray(dogs).astype('float32')
cats = np.asarray(cats).astype('float32')
y_dogs = np.asarray(y_dogs).astype('int32')
y_cats = np.asarray(y_cats).astype('int32')

In [12]:
## fit values between 0 and 1
dogs /= 255
cats /= 255

In [13]:
x = np.concatenate((dogs,cats), axis=0)
y = np.concatenate((y_dogs, y_cats), axis=0)

In [14]:
from sklearn.model_selection import train_test_split
# train set 10000개, test_set 25000개
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=0)

In [15]:
x_train.shape

(20000, 96, 96, 1)

In [16]:
x_test.shape

(5000, 96, 96, 1)

In [17]:
## One-Hot Encoding
y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)

In [21]:
len(classes)

2

In [18]:
y_train.shape

(20000, 2)

In [19]:
y_test.shape

(5000, 2)

In [23]:
IMG_CHANNEL=1
N_CLASSES=2

## Here is our model as a CNN
model = Sequential([
    Conv2D(32, (3,3), padding='same', input_shape=(ROW, COL, IMG_CHANNEL), activation='relu'),
    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(.5),
    Dense(N_CLASSES, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 64)       

In [24]:
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(), metrics=['accuracy'])  # 'adam'을 import 해서 클레스로 씀

In [25]:
## to save checkpoint to use later
modelpath = "model/dogs_vs_cats-cnn-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=2, save_best_only=True)

In [26]:
model.fit(x_train, y_train, batch_size=100, epochs=10, validation_split=0.2,
          callbacks=[checkpointer])

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 226s 14ms/step - loss: 0.7079 - accuracy: 0.5164 - val_loss: 0.6733 - val_accuracy: 0.5723

Epoch 00001: val_loss improved from inf to 0.67334, saving model to model/dogs_vs_cats-cnn-01-0.6733.hdf5
Epoch 2/10
16000/16000 [==============================] - 225s 14ms/step - loss: 0.6460 - accuracy: 0.6254 - val_loss: 0.6121 - val_accuracy: 0.6555

Epoch 00002: val_loss improved from 0.67334 to 0.61214, saving model to model/dogs_vs_cats-cnn-02-0.6121.hdf5
Epoch 3/10
16000/16000 [==============================] - 226s 14ms/step - loss: 0.5852 - accuracy: 0.6923 - val_loss: 0.5530 - val_accuracy: 0.7085

Epoch 00003: val_loss improved from 0.61214 to 0.55300, saving model to model/dogs_vs_cats-cnn-03-0.5530.hdf5
Epoch 4/10
16000/16000 [==============================] - 242s 15ms/step - loss: 0.5404 - accuracy: 0.7268 - val_loss: 0.5448 - val_accuracy: 0.7235

Epoch 00004: val_loss imp

In [27]:
from tensorflow.keras.models import load_model
del model
model = load_model('model/dogs_vs_cats-cnn-07-0.4539.hdf5')

In [28]:
scores = model.evaluate(x_test, y_test, verbose=2)
print('MODEL ACCURACY: %.5f' % scores[1])

5000/5000 - 14s - loss: 0.4374 - accuracy: 0.8020
MODEL ACCURACY: 0.80200
